In [323]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [324]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import math
from ast import literal_eval

In [325]:
frequent_icd9category = ['401','427','276','414','272','250','428','518','285','584']
frequent_icd9code = ['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081']
frequent_icd10category = ['I10', 'I25', 'E78', 'I50', 'I48', 'N17', 'E87', 'E11', 'J96', 'N39']
frequent_icd10code = ['I10', 'I2510', 'I509', 'I4891', 'N179', 'E119', 'E784', 'E785', 'J9690', 'J9600']

In [326]:
def labeltarget(x,frequent_list):
  target=np.zeros(10,dtype="float32")
  for index,code in enumerate(frequent_list):
    if code in x :
      target[index]=1
  return target

In [327]:
train_diagnosis = pd.read_csv("/content/drive/MyDrive/AlgoIntern/Data/Data2.0/discharge_diagnosis_icd_train")
test_diagnosis = pd.read_csv("/content/drive/MyDrive/AlgoIntern/Data/Data2.0/discharge_diagnosis_icd_test")

In [328]:
train_diagnosis

,HADM_ID,discharge_diagnosis,ICD9_CODE,ICD10,ICD9_CATEGORY,ICD10_CATEGORY
0,138999.0,",anoxic brain injury,vericeal bleeding,possibl...","['570', '311', '48242', '3481', '2760', '2874'...","['R569', 'F329', 'E870', 'I498', 'J9600', 'J96...","['518', '054', '570', '456', 'E915', '311', '2...","['R56', 'I49', 'F32', 'J96', 'E87', 'R00']"
1,154986.0,",1.rectosigmoid carcinoma,2.esophagitis/small ...","['V4577', 'V1582', '5533', '1540', '4019', '53...","['Z951', 'Z87891', 'J45909', 'E780', 'J45998',...","['578', '493', '530', '401', 'V15', '154', '27...","['J45', 'Z87', 'Z95', 'I10', 'E78']"
2,151638.0,",likely chronic eosinophilic pneumonia,copd, h...","['4280', '496', '4019', '41401', '5183', 'E879...","['I2510', 'I509', 'J449', 'Y848', 'E780', 'I95...","['458', 'E879', '496', '401', '346', '473', '4...","['I50', 'Y84', 'I95', 'I25', 'J44', 'I10', 'E78']"
3,160956.0,",hypotension secondary to hypovolemia,.,psoria...","['6960', '2550', '32723', '25000', '5718', '27...","['E860', 'E785', 'N179', 'E784', 'I952', 'I958...","['V58', '255', '696', '401', '272', 'E944', '5...","['E86', 'I95', 'E11', 'N17', 'Z79', 'G47', 'I1..."
4,157693.0,",?partial small bowel obstruction,lower gi ble...","['30590', 'V652', '4019', '5789', '5368', '346...","['D62', 'K922', 'J45909', 'D696', 'J45998', 'I...","['536', '578', '305', '493', '401', '346', 'V4...","['D62', 'J45', 'D69', 'K92', 'I10']"
...,...,...,...,...,...,...
33882,146688.0,",acute blood loss anemia/gi bleed,pancytopenia...","['20080', '72981', '28800', 'E8497', 'V1005', ...","['D62', 'N390', 'K922', 'D696', 'J918', 'J9600...","['200', 'E888', '427', '599', '578', '284', '4...","['D62', 'D69', 'N39', 'J91', 'I48', 'K92', 'J9..."
33883,177161.0,",- non-st elevation myocardial infarction,- ac...","['4240', '311', '99664', '7140', '2760', '2500...","['I5032', 'N170', 'Z66', 'I348', 'F329', 'N390...","['806', 'V58', '311', '410', '599', 'E878', '4...","['I34', 'Z66', 'I50', 'I95', 'N39', 'F32', 'N1..."
33884,177705.0,",idiopathic pericarditis w/ pericardial and pl...","['V1301', '49381', '4239', '2724', '5119', '42...","['E785', 'E784', 'J918']","['493', '420', '423', 'V13', '272', '511']","['J91', 'E78']"
33885,195465.0,",subdural hematomas,supratherapeutic inr,atria...","['V4572', '5853', 'V5861', 'E9177', '2449', '7...","['Z7901', 'I4891', 'I129', 'K219', 'E785', 'D6...","['244', '530', 'V58', 'E917', '427', '852', '2...","['D64', 'I12', 'I48', 'N17', 'K21', 'Z79', 'E0..."


In [329]:
train_diagnosis['ICD9_CODE'] = train_diagnosis['ICD9_CODE'].apply(literal_eval)
train_diagnosis['ICD9_CATEGORY'] = train_diagnosis['ICD9_CATEGORY'].apply(literal_eval)
train_diagnosis['ICD10'] = train_diagnosis['ICD10'].apply(literal_eval)
train_diagnosis['ICD10_CATEGORY'] = train_diagnosis['ICD10_CATEGORY'].apply(literal_eval)


In [330]:
train_diagnosis

,HADM_ID,discharge_diagnosis,ICD9_CODE,ICD10,ICD9_CATEGORY,ICD10_CATEGORY
0,138999.0,",anoxic brain injury,vericeal bleeding,possibl...","[570, 311, 48242, 3481, 2760, 2874, 78959, 276...","[R569, F329, E870, I498, J9600, J9690, R001]","[518, 054, 570, 456, E915, 311, 276, 070, 780,...","[R56, I49, F32, J96, E87, R00]"
1,154986.0,",1.rectosigmoid carcinoma,2.esophagitis/small ...","[V4577, V1582, 5533, 1540, 4019, 53019, 2720, ...","[Z951, Z87891, J45909, E780, J45998, I10]","[578, 493, 530, 401, V15, 154, 272, V45, 553, ...","[J45, Z87, Z95, I10, E78]"
2,151638.0,",likely chronic eosinophilic pneumonia,copd, h...","[4280, 496, 4019, 41401, 5183, E8798, 45829, 2...","[I2510, I509, J449, Y848, E780, I952, I10, I9581]","[458, E879, 496, 401, 346, 473, 414, 272, 998,...","[I50, Y84, I95, I25, J44, I10, E78]"
3,160956.0,",hypotension secondary to hypovolemia,.,psoria...","[6960, 2550, 32723, 25000, 5718, 27801, E9320,...","[E860, E785, N179, E784, I952, I9581, E119, I1...","[V58, 255, 696, 401, 272, E944, 584, 729, V85,...","[E86, I95, E11, N17, Z79, G47, I10, E78]"
4,157693.0,",?partial small bowel obstruction,lower gi ble...","[30590, V652, 4019, 5789, 5368, 34690, 29632, ...","[D62, K922, J45909, D696, J45998, I10]","[536, 578, 305, 493, 401, 346, V44, 287, V65, ...","[D62, J45, D69, K92, I10]"
...,...,...,...,...,...,...
33882,146688.0,",acute blood loss anemia/gi bleed,pancytopenia...","[20080, 72981, 28800, E8497, V1005, V443, 5990...","[D62, N390, K922, D696, J918, J9600, I10, J969...","[200, E888, 427, 599, 578, 284, 401, 780, 287,...","[D62, D69, N39, J91, I48, K92, J96, I10]"
33883,177161.0,",- non-st elevation myocardial infarction,- ac...","[4240, 311, 99664, 7140, 2760, 25002, 0413, 42...","[I5032, N170, Z66, I348, F329, N390, E872, I50...","[806, V58, 311, 410, 599, E878, 401, V49, 716,...","[I34, Z66, I50, I95, N39, F32, N17, E87, I21, ..."
33884,177705.0,",idiopathic pericarditis w/ pericardial and pl...","[V1301, 49381, 4239, 2724, 5119, 42091]","[E785, E784, J918]","[493, 420, 423, V13, 272, 511]","[J91, E78]"
33885,195465.0,",subdural hematomas,supratherapeutic inr,atria...","[V4572, 5853, V5861, E9177, 2449, 79092, 53081...","[Z7901, I4891, I129, K219, E785, D649, N179, E...","[244, 530, V58, E917, 427, 852, 272, 403, V45,...","[D64, I12, I48, N17, K21, Z79, E03, E78]"


In [331]:
test_diagnosis['ICD9_CODE'] = test_diagnosis['ICD9_CODE'].apply(literal_eval)
test_diagnosis['ICD9_CATEGORY'] = test_diagnosis['ICD9_CATEGORY'].apply(literal_eval)
test_diagnosis['ICD10'] = test_diagnosis['ICD10'].apply(literal_eval)
test_diagnosis['ICD10_CATEGORY'] = test_diagnosis['ICD10_CATEGORY'].apply(literal_eval)


In [332]:
test_diagnosis

,HADM_ID,discharge_diagnosis,ICD9_CODE,ICD10,ICD9_CATEGORY,ICD10_CATEGORY
0,100009.0,",coronary artery disease,cad-(ami UNK, UNK UNK...","[44021, V1582, 27800, 4019, 4262, 41401, 99604...","[I2510, E669, Z87891, I200, E780, D649, Z9861,...","[278, 401, V58, V85, 250, 440, V15, 414, 426, ...","[D64, E66, I20, E11, Z87, Z98, I25, Z79, I10, ..."
1,100034.0,",coronary artery disease s/p coronary artery b...","[V1582, 6961, 4019, 41401, 412, 25000, 4139, 2...","[I2510, Z87891, E785, E784, I252, E119, I10]","[401, 250, 412, 413, V15, 414, 272, 696]","[E11, Z87, I25, I10, E78]"
2,100044.0,", prematurity, respiratory distress syndrome,,...","[V3001, 7793, V053, 76519, 7717, 7742, 76528, ...","[P002, P590, Z412, Z3801, P220, Z23]","[771, 769, V50, 779, V05, V29, V30, 765, 774]","[P22, Z41, Z23, P00, Z38, P59]"
3,100050.0,",aortic stenosis s/p aortic valve replacement,...","[4280, 41401, 53081, 99791, 5990, 99811, 4241,...","[I2510, N390, I509, I358, Z950, I351, K219, I3...","[997, 530, 424, 414, 998, 287, V45, 428, 427, ...","[I50, D69, N39, I35, I48, K21, Z95, I25]"
4,100065.0,",diabetes mellitus ii with ketoacidosis,.,hype...","[3659, 4019, 25012, 5849, 32723]","[N179, I10, G4733]","[401, 250, 327, 365, 584]","[G47, I10, N17]"
...,...,...,...,...,...,...
14518,199987.0,",s/p motor vehicle crash,injuries:,multiple le...","[8056, 5180, 9190, 8082, 8604, 80709, E8120, 8...","[J9811, J9819]","[808, E812, 805, 807, 860, 919, 518]",[J98]
14519,199988.0,",1. rectal cancer.,2. status post proctocole...","[E8792, 7070, 0389, 5672, 99859, 2720, 5609, 1...","[E780, A419]","[707, 997, 038, 998, 272, 154, E879, 560, 567]","[A41, E78]"
14520,199992.0,", aspiration pneumonia, dysphagia, hypothermia...","[E9352, 78720, 5070, 44382, 0539, 2449, E9363,...","[E870, I498, J690, N179, D649, G4733, R001, E039]","[244, E936, E939, 507, 327, E935, 276, 787, 44...","[J69, D64, I49, N17, E87, G47, R00, E03]"
14521,199993.0,",1. cad status post redo cabg times five and ...","[41031, 4271, 5180, 4240, 42821, 5184, 2760, 5...","[I472, I348, J9819, E870, I340, J9811, J918, I...","[424, 276, 427, 428, 511, 518, 410]","[I34, J91, I48, J98, I47, E87]"


In [333]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train_diagnosis, test_size = 2/7, random_state = 2021, shuffle = True)

In [334]:
train

,HADM_ID,discharge_diagnosis,ICD9_CODE,ICD10,ICD9_CATEGORY,ICD10_CATEGORY
23483,123434.0,", cyanotic episode, stridor, laryngeal edema,,...","[V3001, 4786, 77083, V053, 77989, 53081, V290,...","[P002, Z3801, K219, Z23]","[530, 779, 478, 748, V29, V30, 770, V05]","[Z23, K21, P00, Z38]"
33091,118279.0,",atrial fibrillation,hypertension,obstructive ...","[9971, 71590, V103, 3970, 4019, V5861, 2449, 7...","[G4733, Z7901, I4891, I97790, I498, J45909, I9...","[244, 327, 756, 493, 997, 401, V58, 747, 746, ...","[Y84, J45, I49, I48, R00, I97, Z79, Z85, G47, ..."
17059,112345.0,",1.bilateral subdural hematomas,2.complex skul...","[80502, 80505, 5180, 8730, 9331, 8028, 80321, ...","[J9811, J9819, J189]","[805, 486, E816, 803, 801, 861, 802, 933, E912...","[J18, J98]"
7157,103999.0,",1. type b aortic dissection.,2. endovascular ...","[44101, 44323, 30500, 4471, 44329]",[],"[443, 305, 441, 447]",[]
16295,130174.0,",distal esophagitis and gastritis due to inges...","[9839, 5180, 53501, V6284, 29623, 53019, E9507...","[J9811, J9819]","[530, 296, 983, E950, 535, V62, 518]",[J98]
...,...,...,...,...,...,...
2669,108960.0,",st-elevation myocardial infarction,congestive...","[4280, 78039, 496, 41401, 42820, 2720, 4168, 1...","[I2510, R569, I2789, I509, J449, E780, I272]","[496, 272, 780, 414, 112, 428, 410, 416]","[I50, R56, I27, E78, J44, I25]"
17536,128037.0,",colitis - likely infectious,transaminitis,pul...","[51882, 7931, 3490, 5589, 7242, 5770, 7904, E8...",[],"[724, E879, 793, 790, 558, 577, 349, 791, 518]",[]
6201,125970.0,",st elevation mi","[3963, 41041, 4588, 4260, 41401, V1051]",[I2510],"[458, 396, 414, 426, V10, 410]",[I25]
27989,165330.0,",cholangitis,bacteremia - enterococcus x 2, ps...","[2800, 1970, 2767, 1976, 0417, 7907, 25002, 04...","[I2510, K922, Z950, R7881, N179, E784, E875, J...","[578, 518, 572, 280, 401, V58, 276, 041, 250, ...","[E78, N17, K92, J96, R78, E87, Z79, Z95, I10, ..."


In [335]:
X_train = train['discharge_diagnosis'].values
X_test = test_diagnosis['discharge_diagnosis'].values
y_train_icd9 = train['ICD9_CODE'].values
y_train_icd9cat = train['ICD9_CATEGORY'].values
y_train_icd10 = train['ICD10'].values
y_train_icd10cat = train['ICD10_CATEGORY'].values
y_test_icd9 = test_diagnosis['ICD9_CODE'].values
y_test_icd9cat = test_diagnosis['ICD9_CATEGORY'].values
y_test_icd10 = test_diagnosis['ICD10'].values
y_test_icd10cat = test_diagnosis['ICD10_CATEGORY'].values

In [336]:
X_train[:20]

array([', cyanotic episode, stridor, laryngeal edema,,rule out sepsis, gastroesophageal reflux.',
       ',atrial fibrillation,hypertension,obstructive sleep apnea',
       ',1.bilateral subdural hematomas,2.complex skull fracture involving left squamous temporal and,greater UNK sphenoid,3.cervical spine fx: c1 ring, UNK fracture, anterior,flexion teardrop of c5-c6, unstable body fracture of c7.,4.left sphenoid sinus/maxillary sinus/foramen rotundum fracture,5.right orbitlal fracture,6.bilateral pulmaonry contusions',
       ',1. type b aortic dissection.,2. endovascular repair of aortic dissection with stents,placed in the right common iliac artery, fma, left renal,artery on UNK.,3. stent proximal right common iliac artery on UNK.,1. uncontrolled hypertension, resolved.,2. ethanol withdrawal.',
       ',distal esophagitis and gastritis due to ingestion of foreign,substance,depression,anxiety',
       ',atrial fibrillation,asystolic arrest,congestive heart failure,anoxic brain injury,d

In [337]:
y_train_icd9[:20]

array([list(['V3001', '4786', '77083', 'V053', '77989', '53081', 'V290', '7483']),
       list(['9971', '71590', 'V103', '3970', '4019', 'V5861', '2449', '74740', 'E8798', '49390', '42789', '7464', '75616', '42731', '32723']),
       list(['80502', '80505', '5180', '8730', '9331', '8028', '80321', '80507', '80121', 'E912', '86121', '5185', '486', 'E8161', '78820']),
       list(['44101', '44323', '30500', '4471', '44329']),
       list(['9839', '5180', '53501', 'V6284', '29623', '53019', 'E9507', '53020']),
       list(['5715', '4280', '07070', '78039', '51881', 'V422', '25000', '3481', '42842', '4275', '30391', '2724', '30401', '42731']),
       list(['5967', '59389', '0389', '2762', '5920', '99859', '99811', '5118', '99592', '5185', '9975', '5934']),
       list(['07054', '570', '0389', '431', '4210', '99592', '486', '59381', 'V08', '2875', '5715', '2762', '51881', '79092', 'V4986', '2761', '430', '5724', '3485', '78552', '5849']),
       list(['V3001', '7793', '7708', 'V290', '7861'

In [338]:
y_train_icd9cat = np.array([labeltarget(x, frequent_icd9category) for x in y_train_icd9cat])
y_test_icd9cat = np.array([labeltarget(x, frequent_icd9category) for x in y_test_icd9cat])

In [339]:
y_train_icd9cat[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [340]:
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')).union(set(string.punctuation)) # stopwords + punctuation



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [341]:
def preprocess(text):

  words=word_tokenize(text)
  filtered_sentence = [] 
  # remove stopwords
  for word in words: 
    if word not in stop_words: 
        filtered_sentence.append(word) 
  text = ' '.join(filtered_sentence)
  # lemmatize
  # lemma_word = []
  # wordnet_lemmatizer = WordNetLemmatizer()
  # for w in filtered_sentence:
  #   word1 = wordnet_lemmatizer.lemmatize(w, pos = "n")
  #   word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
  #   word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
  #   lemma_word.append(word3)
  return text



In [342]:
X_train[0]

', cyanotic episode, stridor, laryngeal edema,,rule out sepsis, gastroesophageal reflux.'

In [343]:
# X_train = [preprocess(x) for x in X_train]
# X_test = [preprocess(x) for x in X_test]

In [344]:
X_train[0]

', cyanotic episode, stridor, laryngeal edema,,rule out sepsis, gastroesophageal reflux.'

In [345]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [346]:
# tfidf_vectorizer = TfidfVectorizer(X_train,ngram_range=(1,2),max_df=0.9,min_df=5,token_pattern=r'(\S+)')

In [347]:
tfidf_vectorizer = TfidfVectorizer(max_df = 0.8)

In [348]:
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [349]:
# For multiclass classification
from sklearn.multiclass import OneVsRestClassifier

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from lightgbm import LGBMClassifier

def train_classifier(X_train, y_train, X_valid=None, y_valid=None, C=1.0, model='lr'):
    """
      X_train, y_train — training data
      
      return: trained classifier
      
    """
    
    if model=='lr':
        model = LogisticRegression(solver= "liblinear")
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)
    
    elif model=='svm':
        model = LinearSVC(C=C, penalty='l1', dual=False, loss='squared_hinge')
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)
    
    elif model=='nbayes':
        model = MultinomialNB(alpha=1.0)
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)
        
    elif model=='lda':
        model = LinearDiscriminantAnalysis(solver='svd')
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)

    return model


In [350]:
%time classifier_tfidf = train_classifier(X_train, y_train_icd9cat, C=1.0, model='lr')

CPU times: user 2.15 s, sys: 1.59 s, total: 3.74 s
Wall time: 1.99 s


In [351]:
y_test_predicted_icd9cat = classifier_tfidf.predict(X_test)

In [352]:
# y_test_pred_inversed = mlb.inverse_transform(y_test_predicted_labels_tfidf)
# y_test_inversed = mlb.inverse_transform(y_test)
# for i in range(3):
#     print('Text:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
#         X_test[i],
#         ','.join(y_test_inversed[i]),
#         ','.join(y_test_pred_inversed[i])
#     ))

In [353]:
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score,precision_score
from sklearn.metrics import classification_report

In [354]:
print("Accuracy score for Logistic Regression:")
print(accuracy_score(y_test_icd9cat, y_test_predicted_icd9cat))
print("Hamming loss for Logistic Regression:")
print(hamming_loss(y_test_icd9cat, y_test_predicted_icd9cat))
print(precision_score(y_test_icd9cat, y_test_predicted_icd9cat,average='micro'))
print(recall_score(y_test_icd9cat, y_test_predicted_icd9cat,average='micro'))
print(f1_score(y_test_icd9cat, y_test_predicted_icd9cat,average='micro'))

Accuracy score for Logistic Regression:
0.2087722922261241
Hamming loss for Logistic Regression:
0.19584107966673553
0.7672176909320888
0.4322398329033051
0.5529533809059759


In [355]:
y_train_icd10cat = [labeltarget(x, frequent_icd10category) for x in y_train_icd10cat]
y_test_icd10cat = [labeltarget(x, frequent_icd10category) for x in y_test_icd10cat]

In [356]:
%time classifiericd10cat_tfidf = train_classifier(X_train, y_train_icd10cat, C=1.0, model='lr')

CPU times: user 2.16 s, sys: 1.53 s, total: 3.69 s
Wall time: 2 s


In [357]:
y_test_predicted_icd10cat = classifiericd10cat_tfidf.predict(X_test)

In [358]:
print("Accuracy score for Logistic Regression:")
print(accuracy_score(y_test_icd10cat, y_test_predicted_icd10cat))
print("Hamming loss for Logistic Regression:")
print(hamming_loss(y_test_icd10cat, y_test_predicted_icd10cat))
print(precision_score(y_test_icd10cat, y_test_predicted_icd10cat,average='micro'))
print(recall_score(y_test_icd10cat, y_test_predicted_icd10cat,average='micro'))
print(f1_score(y_test_icd10cat, y_test_predicted_icd10cat,average='micro'))

Accuracy score for Logistic Regression:
0.2646836053157061
Hamming loss for Logistic Regression:
0.16773393926874614
0.7642577030812325
0.40368112682724744
0.5283091937107892


In [359]:
y_train_icd9 = [labeltarget(x, frequent_icd9code) for x in y_train_icd9]
y_test_icd9 = [labeltarget(x, frequent_icd9code) for x in y_test_icd9]

In [360]:
%time classifiericd9_tfidf = train_classifier(X_train, y_train_icd9, C=1.0, model='lr')

CPU times: user 2.15 s, sys: 1.54 s, total: 3.68 s
Wall time: 1.94 s


In [361]:
y_test_predicted_icd9 = classifiericd9_tfidf.predict(X_test)

In [362]:
print("Accuracy score for Logistic Regression:")
print(accuracy_score(y_test_icd9, y_test_predicted_icd9))
print("Hamming loss for Logistic Regression:")
print(hamming_loss(y_test_icd9, y_test_predicted_icd9))
print(precision_score(y_test_icd9, y_test_predicted_icd9,average='micro'))
print(recall_score(y_test_icd9, y_test_predicted_icd9, average = 'micro'))
print(f1_score(y_test_icd9, y_test_predicted_icd9, average ='micro'))


Accuracy score for Logistic Regression:
0.29229498037595536
Hamming loss for Logistic Regression:
0.14828892102182745
0.761943817920853
0.381767747679874
0.5086694652308815


In [363]:
y_train_icd10 = [labeltarget(x, frequent_icd10code) for x in y_train_icd10]
y_test_icd10 = [labeltarget(x, frequent_icd10code) for x in y_test_icd10]

In [364]:
%time classifiericd10_tfidf = train_classifier(X_train, y_train_icd10, C=1.0, model='lr')

CPU times: user 2.21 s, sys: 1.57 s, total: 3.78 s
Wall time: 2.02 s


In [365]:
y_test_predicted_icd10 = classifiericd10_tfidf.predict(X_test)

In [366]:
print("Accuracy score for Logistic Regression:")
print(accuracy_score(y_test_icd10, y_test_predicted_icd10))
print("Hamming loss for Logistic Regression:")
print(hamming_loss(y_test_icd10, y_test_predicted_icd10))
print(precision_score(y_test_icd10, y_test_predicted_icd10, average ='micro'))
print(recall_score(y_test_icd10, y_test_predicted_icd10, average='micro'))
print(f1_score(y_test_icd10, y_test_predicted_icd10, average='micro'))

Accuracy score for Logistic Regression:
0.3303725125662742
Hamming loss for Logistic Regression:
0.15754320732631
0.7519252287237543
0.3741280497789422
0.49965010496850937
